# Test Notebook for Gemini API Translation

Set up an API key with Google at https://aistudio.google.com/app/apikey

The free tier for the API key give us:

- 15 requests per minute
- 1 million input tokens per minute
- 1,500 requests per day

for gemini-1.5-flash and gemini-1.5-8b

We get (for free):

- 2 requests per minute
- 32,000 input tokens per minute
- 50 requests per day

for gemini-1.5-pro

I recommend placing google API key in a secrets.json file (in .gitignore so you don't make it public).


In [1]:
# imports
import json

In [3]:
# Read in google_api_key
with open('../src/secrets.json') as secrets:
    google_api_key = json.load(secrets)['google_api_key']

### Using src/translater.py file


In [4]:
import sys
import os

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the script directory to the Python path
script_dir = os.path.join(parent_dir, "src")
sys.path.append(script_dir)

In [5]:
import translater

In [6]:
t = translater.Translater(secrets_path="../src/secrets.json")

In [8]:
response = t("Let’s race! Hop behind the wheel to put physics and your own driving skills to the test in this next-level racing experience.", "Spanish")

In [14]:
print(response.candidates[0].content.parts[0].text)

¡Participemos en una carrera! Suba al volante para poner a prueba la física y sus propias habilidades de conducción en esta experiencia de carreras de alto nivel.



### Using Google's genai API

Need to 
```
pip install -U google-generativeai
```

if you haven't already. 


In [5]:
import google.generativeai as genai
genai.configure(api_key=google_api_key)

In [6]:
model = genai.GenerativeModel("gemini-1.5-flash") # choose the model

In [7]:
LANGUAGE = 'Spanish'
response = model.generate_content(
    f"Translate the following text to {LANGUAGE} and output only the translated, formal version: 'Let’s race! Hop behind the wheel to put physics and your own driving skills to the test in this next-level racing experience.'"
)

translated_text = response.candidates[0].content.parts[0].text
print(translated_text)

¡Compitamos!  Suba al volante para poner a prueba la física y sus propias habilidades de conducción en esta experiencia de carreras de siguiente nivel.



### Using OpenAI API
```
pip install openai
```
If you don't have it installed. This is very similar just conforming to a different API, and need the base_url for the client object

In [8]:
from openai import OpenAI

client = OpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/"
)

In [9]:
LANGUAGE = "Spanish"
response = client.chat.completions.create(
    model="gemini-1.5-flash",
    n=1,
    messages=[
        {"role": "system", 
         "content": f"You are a helpful accessibility assistant that translates given text and ONLY the given text into {LANGUAGE}. Your output should be the translated text ONLY and use the formal version."},
        {
            "role": "user",
            "content": "Let’s race! Hop behind the wheel to put physics and your own driving skills to the test in this next-level racing experience."
        }
    ]
)

In [10]:
print(response.choices[0].message.content)

¡Vamos a competir! Suba al volante para poner a prueba la física y sus propias habilidades de conducción en esta experiencia de carreras de siguiente nivel.



### Google Cloud Translate
This is probably the direction Discovery World would go if this gets implemented for more organization around the translation process. For right now, can't figure it out without putting in more effort that I do not think is worth it rn.

In [ ]:
from google.cloud import aiplatform

In [ ]:
def translate():
    # Create a client
    endpoint = aiplatform.Ednpoint('projects/gen-lang-client-0091826044/locations/LOCATION/publishers/google/models/cloud-translate-text')
    # TODO where to find LOCATION? Not sure this is worth the effort